In [1]:
!pip install rasterio
!pip install earthpy
!pip install rioxarray
!pip install geopandas
!!pip install pyshp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 12.6 MB/s 
     |████████████████████████████████| 1.0 MB 22.4 MB/s 
     |████████████████████████████████| 16.7 MB 245 kB/s 
     |████████████████████████████████| 6.3 MB 40.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54611 sha256=1c4497689ef2bcd5ebfcd6a7280d44b3829117693ff88bf3d9ff58b4fa4c5f2a
  Stored in directory: /root/.cache/pip/wheels/07/da/9e/1cc57b2e7a29a206893

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting pyshp',
 '  Downloading pyshp-2.3.0-py2.py3-none-any.whl (46 kB)',
 '\x1b[?25l',
 '\x1b[K     |███████                         | 10 kB 22.4 MB/s eta 0:00:01',
 '\x1b[K     |██████████████▏                 | 20 kB 28.1 MB/s eta 0:00:01',
 '\x1b[K     |█████████████████████▎          | 30 kB 33.1 MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████▍   | 40 kB 32.7 MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████████| 46 kB 3.6 MB/s ',
 '\x1b[?25hInstalling collected packages: pyshp',
 'Successfully installed pyshp-2.3.0']

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/EarthEngineToGeoTIFF_v3.py EarthEngineToGeoTIFF_v3.py

In [4]:
import ee
from osgeo import gdal
import rasterio
from rasterio.plot import show as showRasterio
import geopandas as gpd
import shapefile
from shapely.geometry import MultiPoint, Point, Polygon

import pandas as pd
import numpy as np
from PIL import Image
import os
import datetime
from datetime import datetime, timedelta
import random
import matplotlib.pyplot as plt

# my script
from EarthEngineToGeoTIFF_v3 import getSentinalS2SRImage

import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import rioxarray as rxr

random.seed(1234)

In [5]:
# Download file to google drive
!unzip /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/cb_2018_us_state_20m.zip

Archive:  /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/cb_2018_us_state_20m.zip
  inflating: cb_2018_us_state_20m.shp.ea.iso.xml  
  inflating: cb_2018_us_state_20m.shp.iso.xml  
  inflating: cb_2018_us_state_20m.shp  
  inflating: cb_2018_us_state_20m.shx  
  inflating: cb_2018_us_state_20m.dbf  
  inflating: cb_2018_us_state_20m.prj  
 extracting: cb_2018_us_state_20m.cpg  


In [6]:
ee.Authenticate()

ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=3Xv3Du18ib4TLP2PZbGgu60fcQbGo7k8Nki5czd0yvM&tc=dMaN1_eeBbXjMDsUeHQnUNgnCLRLPI1G--eBt1WxXUM&cc=QUUr6c3SE6p-o95SYp44YD0pkqVpE0C7FVGf2KnDNx0

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AdQt8qixm4Q7dyz390JycdrHcQXMk0m4Gi4HLbNge6b0vtMKYj1cN_-bXuc

Successfully saved authorization token.


In [7]:
def imgQuality(dirIM,listIM):
  """
  Check image quality for pixels
  Arguments:
  dirIm (str): directory where images are located
  listIM(list): list of images in directory
  Returns:
  df (DataFrame): returns a dataframe {'name':images, 'cloudQuality':cloudQuality,'missingQuality':Quality}
  """
  df = pd.DataFrame()
  images = []
  cloudQuality = []
  Quality = []
  for im in listIM:
    imd = Image.open(dirIM+im, 'r')
    imA = np.asarray(imd)
    if imA.mean() == 0:
      quality = 0
      cloudquality = 0
      Quality.append(quality)
      cloudQuality.append(cloudquality)
      images.append(im)      
    else:
      vals = imA.shape[0] * imA.shape[1] * imA.shape[2] # denominator
      #quality = 1- (np.bincount(np.reshape(imA, imA.size))[0] / vals) 
      quality = 1 - (np.count_nonzero(imA ==0 )/vals) # number missing pixels/ total number of pixels
      cloudquality = 1- (np.count_nonzero(imA == 1) / vals) # number of white pixels (cloud) / total number pixels
      Quality.append(quality)
      cloudQuality.append(cloudquality)
      images.append(im)
  df = df.append(pd.DataFrame({'name':images, 'cloudQuality':cloudQuality,'missingQuality':Quality}))
  return df



In [8]:
#! cp -r  /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/jpegs006at64 jpegs006at64

In [9]:
# dir006 = "jpegs006at64/"
# l006 = os.listdir(dir006)
# len(l006)

# # Create list of Sinkhole IDs with picture
# tileID = []
# for im in l006:
#     tileID.append(int(im.split("-")[0]))
# len(tileID)

In [10]:
#sh1 = pd.read_excel("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/Florida_Subsidence_Incident_Reports.xlsx", sheet_name = 0)
sh1 = pd.read_excel("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/sinkhole_10_years.xlsx")
sh1.head()

,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,TRUE_SINK,LONGDD,LATDD,...,LIMVIS,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_
0,-82.682258,28.360988,67,14-758,2015/11/03 00:00:00+00,2012/06/20 00:00:00+00,2012,U,-82.682255,28.360983,...,U,U,R,Y,Unknown,Unknown,UNKNOWN,"40' dia, 20' deep; sinkhole destroyed back hal...",NaN,NaN
1,-84.865330,29.744150,106,49-001,2015/10/30 00:00:00+00,2012/08/25 00:00:00+00,2012,U,-84.865326,29.744144,...,N,U,R,Y,Repaired,Unknown,UNKNOWN,SWO# 2012-6049: subsidence occurred in the roa...,Location confirmed via Google Maps image on in...,NaN
2,-82.991917,30.288370,124,37-435,2014/12/08 00:00:00+00,2012/06/27 00:00:00+00,2012,U,-82.991913,30.288364,...,U,U,U,U,Unknown,Unknown,NaN,NaN,Report 37-436 says 4 sinkholes located near ch...,NaN
3,-81.945712,27.946343,126,16-896,2021/07/30 00:00:00+00,2014/02/14 00:00:00+00,2014,U,-81.945708,27.946338,...,N,U,U,U,Repaired,Unknown,NaN,Sinkhole was caused by digging near a retentio...,NaN,NaN
4,-81.745600,28.787240,144,11-559,2021/08/05 00:00:00+00,2015/08/24 00:00:00+00,2015,U,-81.745597,28.787234,...,U,U,U,U,Unknown,Unknown,NaN,Lake County Dispatch reports a 4 x 4 sinkhole ...,Report #: 2015-6132,NaN


In [11]:
dfselect = sh1.copy()
dfselect.head()

,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,TRUE_SINK,LONGDD,LATDD,...,LIMVIS,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_
0,-82.682258,28.360988,67,14-758,2015/11/03 00:00:00+00,2012/06/20 00:00:00+00,2012,U,-82.682255,28.360983,...,U,U,R,Y,Unknown,Unknown,UNKNOWN,"40' dia, 20' deep; sinkhole destroyed back hal...",NaN,NaN
1,-84.865330,29.744150,106,49-001,2015/10/30 00:00:00+00,2012/08/25 00:00:00+00,2012,U,-84.865326,29.744144,...,N,U,R,Y,Repaired,Unknown,UNKNOWN,SWO# 2012-6049: subsidence occurred in the roa...,Location confirmed via Google Maps image on in...,NaN
2,-82.991917,30.288370,124,37-435,2014/12/08 00:00:00+00,2012/06/27 00:00:00+00,2012,U,-82.991913,30.288364,...,U,U,U,U,Unknown,Unknown,NaN,NaN,Report 37-436 says 4 sinkholes located near ch...,NaN
3,-81.945712,27.946343,126,16-896,2021/07/30 00:00:00+00,2014/02/14 00:00:00+00,2014,U,-81.945708,27.946338,...,N,U,U,U,Repaired,Unknown,NaN,Sinkhole was caused by digging near a retentio...,NaN,NaN
4,-81.745600,28.787240,144,11-559,2021/08/05 00:00:00+00,2015/08/24 00:00:00+00,2015,U,-81.745597,28.787234,...,U,U,U,U,Unknown,Unknown,NaN,Lake County Dispatch reports a 4 x 4 sinkhole ...,Report #: 2015-6132,NaN


In [12]:
tiles = pd.read_excel("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/sentinel_tiles.xlsx")
tilesFL = tiles[(tiles["State"]=="FL")]
tilesFL.head()

,State,Row,TILE_ID,MULTIPOLIGON,Comments
0,FL,1,16RDV,MULTIPOLYGON(((-88.0548969859722 31.6311891453...,Shared between Alabama and Florida. Mostly Con...
1,FL,1,16REV,MULTIPOLYGON(((-87.0002109102755 31.6355471130...,Shared between Alabama and Florida. Mostly Con...
2,FL,1,16RFV,MULTIPOLYGON(((-85.9455247432055 31.6311926291...,Shared between Alabama and Florida. Mostly Con...
7,FL,2,16RDU,MULTIPOLYGON(((-88.0449388796329 30.7288635166...,Half is ocean - half into Alabama
8,FL,2,16REU,MULTIPOLYGON(((-87.0002089188631 30.7330700317...,half is ocean


In [13]:
def get_polygon(tiles, tile_id):
    """
    Get coordinates of the polygon

    Parameters:
    tiles(DataFrame): Dataframe with tiles information
    tile_id(str): tile id from sentinel-2

    Returns:
    coords(tuple): polygon coordinates in tuple

    """
    mp = tiles[(tiles["TILE_ID"]==tile_id)][["MULTIPOLIGON"]].values
    mp1 = mp[0][0][15:]
    mp1 = mp1[0:len(mp1)-3]
    mp1 = mp1.split(",")
    
    coords = []
    for e1 in mp1:
        sp1 = e1.split(" ")
        coords.append([float(sp1[0]), float(sp1[1])])
    
    return (coords)

In [14]:
def convertdate(date, formatD):
  """
  Convert date from string format to specified date format
  Parameters:
  date(str): date string
  formatD(str): format for date

  Returns:
  datetime(date): formatted datetime
  """
    
  if type(date) == str:
    d = date[0:10]
    return(datetime.strptime(d,"%Y/%m/%d"))
  else:
    return("NaN")

In [15]:
def split_tiles(xdiv, ydiv, tiff_path,name, dirsplit,poi):
    """
    Splits geotiff equally into xdiv x ydiv images
  
    Parameters:
    xdiv (int): number of divisions on x axis
    ydiv (int): number of divisions on y axis
    tiff_path (str): location of RGB tiff 'C:\\Users\\username\\file'
    name (str): name of rgb geotiff file i.e. 'S2B_MSIL2A_20220508T155819_N0400_R097_T17RLN_20220508T205707'
    dirsplit (str):directory where split files are going to be stored  i.e. "tifsat64/"
    poi (tuple):  point of interest (i.e. location of sinkhole) (latitude,longitude)
    Returns:
    coordinates(dict): return {'name':names, 'label':labels, 'corners':corners}
  
    """

    dem = gdal.Open(tiff_path)
    gt = dem.GetGeoTransform()

    # get coordinates of upper left corner
    xmin = gt[0]
    ymax = gt[3]
    res = gt[1]

    # determine total length of raster
    xlen = res * dem.RasterXSize
    ylen = res * dem.RasterYSize

    # number of tiles in x and y direction
    xdiv = xdiv
    ydiv = ydiv

    # size of a single tile
    xsize = xlen/xdiv
    ysize = ylen/ydiv

    # create lists of x and y coordinates
    xsteps = [xmin + xsize * i for i in range(xdiv+1)]
    ysteps = [ymax - ysize * i for i in range(ydiv+1)]
    
    # empty dictionary for coordinates
    names = []
    corners = []
    labels = []
    
    # loop over min and max x and y coordinates
    for i in range(xdiv):
        for j in range(ydiv):
            xmin = xsteps[i]
            xmax = xsteps[i+1]
            ymax = ysteps[j]
            ymin = ysteps[j+1]
            # get coordinates of corners
#             corner1 = convert_coordinates(xmin, ymin)
#             corner2 = convert_coordinates(xmin,ymax)
#             corner3 = convert_coordinates(xmax, ymax)
#             corner4 = convert_coordinates(xmax, ymin)

            corner1 = [xmin, ymin]
            corner2 = [xmin,ymax]
            corner3 = [xmax, ymax]
            corner4 = [xmax, ymin]
            

            corner = [corner1, corner2,corner3,corner4]
            corners.append(corner)
            
            #identify if sinkhole is in subtile
            if (poi[0]>=corner1[0]) & (poi[0]<=corner3[0]) & (poi[1]>=corner1[1]) & (poi[1]<=corner3[1]):
                label_name ='_sinkhole'
                label = 1
            else:
                label_name = ''
                label = 0
            names.append(name+"_"+str(i)+"_"+str(j)+label_name+".jpeg")
            labels.append(label)
            # use gdal warp
            gdal.Warp(dirsplit+name+"_"+str(i)+"_"+str(j)+label_name+".tif", dem, 
                      outputBounds = (xmin, ymin, xmax, ymax), dstNodata = -9999)
            # or gdal translate to subset the input raster
#             gdal.Translate(name+str(i)+str(j)+".tif", dem, projWin = (xmin, ymax, xmax, ymin), xRes = res, yRes = -res)
    # dem
    dem = None
    
    return {'name':names, 'label':labels, 'corners':corners}
    

In [16]:
dfselect

,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,TRUE_SINK,LONGDD,LATDD,...,LIMVIS,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_
0,-82.682258,28.360988,67,14-758,2015/11/03 00:00:00+00,2012/06/20 00:00:00+00,2012,U,-82.682255,28.360983,...,U,U,R,Y,Unknown,Unknown,UNKNOWN,"40' dia, 20' deep; sinkhole destroyed back hal...",NaN,NaN
1,-84.865330,29.744150,106,49-001,2015/10/30 00:00:00+00,2012/08/25 00:00:00+00,2012,U,-84.865326,29.744144,...,N,U,R,Y,Repaired,Unknown,UNKNOWN,SWO# 2012-6049: subsidence occurred in the roa...,Location confirmed via Google Maps image on in...,NaN
2,-82.991917,30.288370,124,37-435,2014/12/08 00:00:00+00,2012/06/27 00:00:00+00,2012,U,-82.991913,30.288364,...,U,U,U,U,Unknown,Unknown,NaN,NaN,Report 37-436 says 4 sinkholes located near ch...,NaN
3,-81.945712,27.946343,126,16-896,2021/07/30 00:00:00+00,2014/02/14 00:00:00+00,2014,U,-81.945708,27.946338,...,N,U,U,U,Repaired,Unknown,NaN,Sinkhole was caused by digging near a retentio...,NaN,NaN
4,-81.745600,28.787240,144,11-559,2021/08/05 00:00:00+00,2015/08/24 00:00:00+00,2015,U,-81.745597,28.787234,...,U,U,U,U,Unknown,Unknown,NaN,Lake County Dispatch reports a 4 x 4 sinkhole ...,Report #: 2015-6132,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,-81.222853,28.631983,4127,77-719,NaN,2018/07/14 00:00:00+00,2018,U,-81.222850,28.631978,...,Y,U,R,Y,Planned,Unknown,NaN,sinkhole reported in road of Wembley Place nea...,SWO# 2018-4337,NaN
851,-83.089320,30.297916,4134,37-417,2016/02/29 00:00:00+00,2012/06/29 00:00:00+00,2012,U,-83.089316,30.297910,...,U,U,U,U,Unknown,Unknown,NaN,NaN,"Cannot locate exact address, did not move",NaN
852,-81.810803,28.530095,4137,11-557,NaN,2014/10/05 00:00:00+00,2014,U,-81.810800,28.530090,...,U,U,U,N,Unknown,Unknown,NaN,The sinkhole is currently about 10 feet wide a...,NaN,NaN
853,-84.234614,30.299485,4138,55-751,NaN,2014/12/24 00:00:00+00,2014,U,-84.234610,30.299479,...,N,N,R,N,Unknown,Unknown,NaN,Sinkholes formed overnight after a heavy rainf...,NaN,NaN


In [17]:
dfselect["DateD"] = dfselect.apply(lambda row: convertdate(row["EVENT_DATE"], "%Y-%m-%d"), axis=1)
dfselect = dfselect[dfselect['DateD']>='2019-11-01']
dfselect.head()

,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,TRUE_SINK,LONGDD,LATDD,...,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_,DateD
83,-81.932475,28.944928,428,11-581,2019/12/17 00:00:00+00,2019/12/11 00:00:00+00,2019,N,-81.932472,28.944923,...,U,U,Y,Unknown,Unknown,NaN,Residence 20 ft away is threatened,SWO# 2019-6915,NaN,2019-12-11
84,-82.161323,29.208174,429,36-797,2020/01/06 00:00:00+00,2020/01/05 00:00:00+00,2020,N,-82.161320,29.208169,...,U,U,U,Unknown,Unknown,NaN,"1,000,000 gallons of treated water from the re...",SWO# 2020-87,NaN,2020-01-05
86,-82.625674,30.192630,431,29-507,2019/12/05 00:00:00+00,2019/12/05 00:00:00+00,2019,N,-82.625670,30.192624,...,U,U,Y,Unknown,Unknown,NaN,"10 Families impacted, no major roadways closed",SWO# 2019-6813,NaN,2019-12-05
89,-86.093305,30.682650,434,60-503,2021/07/14 00:00:00+00,2020/02/18 00:00:00+00,2020,U,30.682749,-86.093285,...,U,U,U,Unknown,Unknown,NaN,SWP# 2020-1034,NaN,NaN,2020-02-18
90,-86.093300,30.682644,435,55-763,2021/07/12 00:00:00+00,2020/05/31 00:00:00+00,2020,U,-84.232400,30.299860,...,N,U,N,Unknown,Unknown,NaN,pool backwash was the source of creation.,NaN,10072 Green Fountain Rd,2020-05-31


In [18]:
def create_dir(tfdir):
    """
    Creates a new directory if it does not exist
  
    Parameters:
    tfdir (str): location and name of new directory

    """
    if os.path.exists(tfdir)==False:
        os.makedirs(tfdir)
        print('{} directory created'.format(tfdir))
    else:
        print('{} directory already exists'.format(tfdir))

In [19]:
tifdir = "image20_10yr/"
tifdir64 ='tifs200at64_10yr'
create_dir("image20_10yr")
create_dir('tifs200at64_10yr')
dirsplit = "tifs200at64_10yr/"

image20_10yr directory created
tifs200at64_10yr directory created


In [20]:
def get_satellite_images(sh_target, sze, folder, num=3,td_1=365, td_2=60):
    td = timedelta(td_1)
    td2 = timedelta(td_2)
    data = pd.DataFrame()
    #label =1
    fnames, labels, sh_IDs, lons, lats, sds, imgnums = [], [], [], [], [], [], []
    for index, row in sh_target.iterrows():
        lat = row["Y"]
        lon = row["X"]
        sh_ID = row["OBJECTID"]
        dateD = row["DateD"]
        dateStr = row["EVENT_DATE"][0:10]
        startDate = dateD - td
        endDate = startDate + td2

        start_date = startDate.strftime("%Y-%m-%d")
        end_date = endDate.strftime("%Y-%m-%d")
        
        print(start_date, end_date)
        
        for z in list(range(num+1)):
          fname = str(sh_ID)+"-"+start_date+"-"+end_date+"-"+str(sze*1000)+str(num)
          fnames.append(fname+'-'+str(z)+'.tif')
          sh_IDs.append(sh_ID)
          lons.append(lon)
          lats.append(lat)
          sds.append(startDate)
          imgnums.append(z)
          #labels.append(label)
        _ = getSentinalS2SRImage(lon, lat, sze, folder+fname, start_date, 
                                end_date, j=num)
        
    data = data.append(pd.DataFrame({"name": fnames, 
                              'imgnum': imgnums,
                              #'label' : labels,
                              "ID": sh_IDs, 
                              "lon": lons, 
                              "lat":lats, 
                              "start_date": sds}))
    return data


In [21]:
def splits_loop():
    """
    Loops through all tif files and splits them into subtiles
    
    Returns:
    coordinates (DataFrame): {'name':names, 'label':labels, 'corners':corners}
    """
    splitN = 35
    num = 0

    for i in os.listdir(tifdir):
        incident = sh1[sh1['OBJECTID'] == float(i.split('-')[0])]
        poi = (incident.iloc[0].X, incident.iloc[0].Y)
        fname = i
        file = split_tiles(splitN,splitN, tifdir+fname, fname, dirsplit,poi)
        if num == 0:
            coordinates = pd.DataFrame(file)
        else:
            coordinates = coordinates.append(pd.DataFrame(file))
        num +=1
    return coordinates

In [22]:
def get_centroid(corners):    
    """
    Returns centroid coordinates of a Polygon
  
    Parameters:
    corners (list): list of tile coordinates [[xmin, ymin],[xmin,ymax],[xmax, ymax],[xmax, ymin]]
    Returns:
    coordinates(list): return [center.x, center.y]
  
    """
    center = Polygon(corners).centroid
    return [center.x, center.y]

In [23]:
IMGs=get_satellite_images(dfselect, 0.20, "image20_10yr/", num=0,td_1=1, td_2=60)

2019-12-10 2020-02-08
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/428-2019-12-10-2020-02-08-200.00-0.tif
2020-01-04 2020-03-04
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/429-2020-01-04-2020-03-04-200.00-0.tif
2019-12-04 2020-02-02
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/431-2019-12-04-2020-02-02-200.00-0.tif
2020-02-17 2020-04-17
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/434-2020-02-17-2020-04-17-200.00-0.tif
2020-05-30 2020-07-29
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/435-2020-05-30-2020-07-29-200.00-0.tif
2020-04-30 2020-06-29
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/541-2020-04-30-2020-06-29-200.00-0.tif
2021-06-23 2021-08-22
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/542-2021-06-23-2021-08-22-200.00-0.tif
2021-04-22 2021-06-21
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr

/content/EarthEngineToGeoTIFF_v3.py:128: RuntimeWarning: invalid value encountered in true_divide
  dest.write( (np.clip(B4.read(1), p2, p98) - p2)/(p98 - p2)*255, 1)
/content/EarthEngineToGeoTIFF_v3.py:129: RuntimeWarning: invalid value encountered in true_divide
  dest.write( (np.clip(B3.read(1), p2, p98) - p2)/(p98 - p2)*255, 2)
/content/EarthEngineToGeoTIFF_v3.py:130: RuntimeWarning: invalid value encountered in true_divide
  dest.write( (np.clip(B2.read(1), p2, p98) - p2)/(p98 - p2)*255, 3)


2021-02-25 2021-04-26
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/1078-2021-02-25-2021-04-26-200.00-0.tif
2021-02-05 2021-04-06
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/1079-2021-02-05-2021-04-06-200.00-0.tif
2020-12-08 2021-02-06
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/1080-2020-12-08-2021-02-06-200.00-0.tif
2020-10-27 2020-12-26
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/1081-2020-10-27-2020-12-26-200.00-0.tif
2020-10-19 2020-12-18
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/1082-2020-10-19-2020-12-18-200.00-0.tif
2021-07-26 2021-09-24
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/1083-2021-07-26-2021-09-24-200.00-0.tif
2021-08-07 2021-10-06
0
Creating 3-band GeoTIFF image ... 
Actual filename is image20_10yr/1086-2021-08-07-2021-10-06-200.00-0.tif
2021-08-12 2021-10-11
0
Creating 3-band GeoTIFF image ... 
Actual filename is image

In [24]:
coordinates = splits_loop()

In [25]:
def getcornercoordinates(path):
  """
  Get coordinates for the four corners of a tif file
  
  Parameters:
  path(str): path where tif files are located
  Returns:
  df (DataFrame): with name and tuple of coordinates

  """
  names = []
  polygons = []
  files = os.listdir(path)
  for name in files:
    names.append(name)
    ds = gdal.Open(path+name)
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5] 
    maxx = gt[0] + width*gt[1] + height*gt[2]
    maxy = gt[3]

    corner1 = (minx,miny)
    corner2 = (minx,maxy)
    corner3 = (maxx,maxy)
    corner4 = (maxx,miny)
    polygons.append([corner1,corner2, corner3, corner4])
  return pd.DataFrame({'name':names, 'polygon':polygons})
    

In [26]:
def create_csv_shapefile(coordinates, name='subtile_coordinates'):
    """
    Saves a csv and shapefile locally, returns Geopandas and Pandas DataFrames
  
    Parameters:
    coordinates(DataFrame): {'name':names, 'label':labels, 'corners':corners}
    
    Returns:
    (df, coordinates): df is the shapefile and coordinates in the Pandas DataFrame
  
    """
    
    # Find centroid coordinates
    coordinates['centroid_x'] = coordinates.apply(lambda row: get_centroid(row['corners'])[0],axis=1)
    coordinates['centroid_y'] = coordinates.apply(lambda row: get_centroid(row['corners'])[1],axis=1)
    # convert corners to geometry shape for geopandas
    coordinates['geometry'] = coordinates.apply(lambda row: Polygon(row['corners']),axis=1)
    coordinates = coordinates[['name','label','centroid_x','centroid_y','geometry']]
    
    df = gpd.GeoDataFrame(coordinates, geometry='geometry')
  
    # Save CSV
    coordinates.to_csv('/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/'+name+'.csv',index=None)
    # Save Shapefile
    df.to_file('/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/'+name+'.shp')
    return df, coordinates
    

In [27]:
df, coordinates = create_csv_shapefile(coordinates,name='subtiles_fl_60_group2')

In [28]:
#!cp -av  "/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/tifs200at64" "/content/tifs200at64"

In [29]:
def create_dir(tfdir):
    """
    Creates a new directory if it does not exist
  
    Parameters:
    tfdir (str): location and name of new directory

    """
    if os.path.exists(tfdir)==False:
        os.makedirs(tfdir)
        print('{} directory created'.format(tfdir))
    else:
        print('{} directory already exists'.format(tfdir))


In [30]:
def remove_xmls(jpegdir):
    """
    Deletes xml files in directory
  
    Parameters:
    jpegdir (str): path to delete xml files associated with jpeg files
  
    """
    for file in os.listdir(jpegdir):
        if ".xml" in file:
            os.remove(jpegdir+file)
    print('xml files deleted')

In [31]:
def selectQualityImgs(jpegdir, randomIMGs, threshold = .90, random=True):
  """
  Return DataFrame with the highest quality images identified based on a specified quality criteria

  Parameters:
    jpegdir(str): specifies directory where jpeg images are stored
    randomIMGs(DataFrame): DataFrame generated from the get_random_images function
    threshold(float): specify a number between 0 to 1 for the upper bound threshold for what is acceptable quality

  Returns:
    randomIMGs_cleaned(DataFrame)

  """
  # get list of jpeg imgs in the directory
  l006 = os.listdir(jpegdir)
  check = imgQuality(jpegdir, l006)
  # identify images that do not meet quality threshold
  poor_quality_imgs = check[check['missingQuality']<threshold]
  # get rows that have good quality
  clean_images = check[~check['name'].isin(list(poor_quality_imgs.name))].reset_index(drop=True)
  # get columns for identifying the first image for each sinkhole incident (OBJECTID and imgnum)
  clean_images['OBJECTID'] = clean_images.apply(lambda row: int(row['name'].split('-')[0]), axis=1)
  if random:
    clean_images['imgnum'] = clean_images.apply(lambda row: int(row['name'].split('-')[9].replace('.jpeg','')), axis=1)
  else: 
    clean_images['imgnum'] = clean_images.apply(lambda row: int(row['name'].split('-')[8].replace('.jpeg','')), axis=1)
  # find the first image based on cloud coverage from query
  clean_imgs = clean_images.groupby(by=['OBJECTID']).min('imgnum').reset_index()
  clean_imgs = clean_imgs[['OBJECTID','imgnum']]

  #identify these rows with first images
  match = pd.merge(clean_images,clean_imgs, on=['OBJECTID', 'imgnum'], how='inner')
  # get list of them
  names_clean = list(match.name)
  # rename to join with csv DataFrame
  names_clean = [i.replace('.jpeg','.tif') for i in names_clean]
  randomIMGs_cleaned = randomIMGs[randomIMGs['name'].isin(names_clean)].reset_index(drop=True)
  return randomIMGs_cleaned


In [32]:
def convert2jpeg(tifdir, jpegdir,filelist):
    """
    Saves tiff file as jpeg
  
    Parameters:
    tifdir (str): path of tif files
    jpegdir (str): path to save jpeg files
  
    """
    
    options_list = [
        '-ot Byte',
        '-of JPEG',
        '-b 3',
        '-b 2',
        '-b 1',
        '-scale'
    ]           

    options_string = " ".join(options_list)

    for filetif in filelist:
      jpegname = filetif.replace(".tif", ".jpeg")
      print(filetif)        
      gdal.Translate(
              jpegdir+jpegname,
              tifdir+filetif,
              options=options_string
          )

In [33]:
import pandas as pd
gdf = df.copy()

In [34]:
gdf['id'] = gdf.apply(lambda row: int(row['name'].split('-')[0]),axis=1)

In [35]:
positive = gdf[gdf['label']==1].reset_index(drop=True)

In [36]:
positive

,name,label,centroid_x,centroid_y,geometry,id
0,1078-2021-02-25-2021-04-26-200.00-0.tif_17_17_...,1,-80.084628,26.260585,"POLYGON ((-80.08749 26.25773, -80.08749 26.263...",1078
1,1088-2021-11-07-2022-01-06-200.00-0.tif_17_17_...,1,-85.594624,30.565627,"POLYGON ((-85.59748 30.56277, -85.59748 30.568...",1088
2,429-2020-01-04-2020-03-04-200.00-0.tif_17_17_s...,1,-82.161308,29.208182,"POLYGON ((-82.16417 29.20532, -82.16417 29.211...",429
3,2405-2022-03-15-2022-05-14-200.00-0.tif_17_17_...,1,-82.166159,29.174361,"POLYGON ((-82.16902 29.17150, -82.16902 29.177...",2405
4,2012-2020-11-23-2021-01-22-200.00-0.tif_17_17_...,1,-81.850716,26.488892,"POLYGON ((-81.85357 26.48603, -81.85357 26.491...",2012
...,...,...,...,...,...,...
108,1083-2021-07-26-2021-09-24-200.00-0.tif_17_17_...,1,-82.792285,27.814087,"POLYGON ((-82.79514 27.81123, -82.79514 27.816...",1083
109,3004-2021-10-25-2021-12-24-200.00-0.tif_17_17_...,1,-82.951781,29.886276,"POLYGON ((-82.95464 29.88342, -82.95464 29.889...",3004
110,3293-2021-10-08-2021-12-07-200.00-0.tif_17_17_...,1,-82.583472,28.904642,"POLYGON ((-82.58633 28.90178, -82.58633 28.907...",3293
111,2461-2021-10-28-2021-12-27-200.00-0.tif_17_17_...,1,-81.533431,27.733643,"POLYGON ((-81.53629 27.73078, -81.53629 27.736...",2461


In [37]:
negative = gdf[gdf['label']==0].reset_index(drop=True)

In [38]:
negative_random = pd.DataFrame()
for id in list(set(gdf.id)):
  new_df = negative[negative['id'] == id]
  sample = new_df.sample(n=1, random_state=2)
  negative_random = negative_random.append(sample)

In [39]:
negative_random.reset_index(drop=True, inplace=True)

In [40]:
final = positive.append(negative_random)
final

,name,label,centroid_x,centroid_y,geometry,id
0,1078-2021-02-25-2021-04-26-200.00-0.tif_17_17_...,1,-80.084628,26.260585,"POLYGON ((-80.08749 26.25773, -80.08749 26.263...",1078
1,1088-2021-11-07-2022-01-06-200.00-0.tif_17_17_...,1,-85.594624,30.565627,"POLYGON ((-85.59748 30.56277, -85.59748 30.568...",1088
2,429-2020-01-04-2020-03-04-200.00-0.tif_17_17_s...,1,-82.161308,29.208182,"POLYGON ((-82.16417 29.20532, -82.16417 29.211...",429
3,2405-2022-03-15-2022-05-14-200.00-0.tif_17_17_...,1,-82.166159,29.174361,"POLYGON ((-82.16902 29.17150, -82.16902 29.177...",2405
4,2012-2020-11-23-2021-01-22-200.00-0.tif_17_17_...,1,-81.850716,26.488892,"POLYGON ((-81.85357 26.48603, -81.85357 26.491...",2012
...,...,...,...,...,...,...
108,2010-2021-06-26-2021-08-25-200.00-0.tif_13_31....,0,-82.560656,28.790897,"POLYGON ((-82.56351 28.78804, -82.56351 28.793...",2010
109,2011-2020-11-23-2021-01-22-200.00-0.tif_13_31....,0,-82.673629,28.147155,"POLYGON ((-82.67649 28.14430, -82.67649 28.150...",2011
110,2012-2020-11-23-2021-01-22-200.00-0.tif_13_31....,0,-81.873589,26.408834,"POLYGON ((-81.87645 26.40598, -81.87645 26.411...",2012
111,2013-2021-07-10-2021-09-08-200.00-0.tif_13_31....,0,-82.447164,29.035679,"POLYGON ((-82.45002 29.03282, -82.45002 29.038...",2013


In [41]:
filelist = [i.replace('jpeg','tif') for i in list(final['name'])]

In [42]:
final['name'].iloc[0]

'1078-2021-02-25-2021-04-26-200.00-0.tif_17_17_sinkhole.jpeg'

In [43]:
tifdir = "tifs200at64_10yr/"
create_dir('jpegs200at64_10yr')
jpegdir = "jpegs200at64_10yr/" 

jpegs200at64_10yr directory created


In [44]:
convert2jpeg(tifdir, jpegdir,filelist)

1078-2021-02-25-2021-04-26-200.00-0.tif_17_17_sinkhole.tif
1088-2021-11-07-2022-01-06-200.00-0.tif_17_17_sinkhole.tif
429-2020-01-04-2020-03-04-200.00-0.tif_17_17_sinkhole.tif
2405-2022-03-15-2022-05-14-200.00-0.tif_17_17_sinkhole.tif
2012-2020-11-23-2021-01-22-200.00-0.tif_17_17_sinkhole.tif
2063-2020-06-06-2020-08-05-200.00-0.tif_17_17_sinkhole.tif
2735-2022-03-15-2022-05-14-200.00-0.tif_17_17_sinkhole.tif
3292-2021-08-04-2021-10-03-200.00-0.tif_17_17_sinkhole.tif
543-2021-04-22-2021-06-21-200.00-0.tif_17_17_sinkhole.tif
541-2020-04-30-2020-06-29-200.00-0.tif_17_17_sinkhole.tif
2459-2021-09-03-2021-11-02-200.00-0.tif_17_17_sinkhole.tif
3931-2021-08-05-2021-10-04-200.00-0.tif_17_17_sinkhole.tif
2334-2020-04-30-2020-06-29-200.00-0.tif_17_17_sinkhole.tif
2346-2021-08-02-2021-10-01-200.00-0.tif_17_17_sinkhole.tif
559-2022-03-25-2022-05-24-200.00-0.tif_17_17_sinkhole.tif
2347-2021-08-15-2021-10-14-200.00-0.tif_17_17_sinkhole.tif
1082-2020-10-19-2020-12-18-200.00-0.tif_17_17_sinkhole.tif
5

In [45]:
def remove_xmls(jpegdir):
    """
    Deletes xml files in directory
  
    Parameters:
    jpegdir (str): path to delete xml files associated with jpeg files
  
    """
    for file in os.listdir(jpegdir):
        if ".xml" in file:
            os.remove(jpegdir+file)
    print('xml files deleted')

In [46]:
remove_xmls(jpegdir)

xml files deleted


In [47]:
%cp -av "/content/jpegs200at64_10yr" "/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/"


'/content/jpegs200at64_10yr/1078-2021-02-25-2021-04-26-200.00-0.jpeg_17_17_sinkhole.jpeg' -> '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/jpegs200at64_10yr/1078-2021-02-25-2021-04-26-200.00-0.jpeg_17_17_sinkhole.jpeg'
'/content/jpegs200at64_10yr/1088-2021-11-07-2022-01-06-200.00-0.jpeg_17_17_sinkhole.jpeg' -> '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/jpegs200at64_10yr/1088-2021-11-07-2022-01-06-200.00-0.jpeg_17_17_sinkhole.jpeg'
'/content/jpegs200at64_10yr/429-2020-01-04-2020-03-04-200.00-0.jpeg_17_17_sinkhole.jpeg' -> '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/jpegs200at64_10yr/429-2020-01-04-2020-03-04-200.00-0.jpeg_17_17_sinkhole.jpeg'
'/content/jpegs200at64_10yr/2405-2022-03-15-2022-05-14-200.00-0.jpeg_17_17_sinkhole.jpeg' -> '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/jpegs200at64_10yr/2405-2022-03-15-2022-05-14-200.00-0.jpeg_17_17_sinkhole.jpeg'
'/content/jpegs200at64_10yr/2012-2020-11-2

In [48]:
create_dir('predictions')
create_dir('predictions/unlabeled/')
! cp -r /content/jpegs200at64_10yr/* predictions/unlabeled/

print('{} files in predictions/unlabeled directory'.format(len(os.listdir('predictions/unlabeled'))))

predictions directory created
predictions/unlabeled/ directory created
226 files in predictions/unlabeled directory


In [49]:
!cp predictions.csv /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/predictions.csv

cp: cannot stat 'predictions.csv': No such file or directory


In [50]:
import pandas as pd
#final = pd.read_csv("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/final_200_subtiles.csv")

In [51]:
final

,name,label,centroid_x,centroid_y,geometry,id
0,1078-2021-02-25-2021-04-26-200.00-0.tif_17_17_...,1,-80.084628,26.260585,"POLYGON ((-80.08749 26.25773, -80.08749 26.263...",1078
1,1088-2021-11-07-2022-01-06-200.00-0.tif_17_17_...,1,-85.594624,30.565627,"POLYGON ((-85.59748 30.56277, -85.59748 30.568...",1088
2,429-2020-01-04-2020-03-04-200.00-0.tif_17_17_s...,1,-82.161308,29.208182,"POLYGON ((-82.16417 29.20532, -82.16417 29.211...",429
3,2405-2022-03-15-2022-05-14-200.00-0.tif_17_17_...,1,-82.166159,29.174361,"POLYGON ((-82.16902 29.17150, -82.16902 29.177...",2405
4,2012-2020-11-23-2021-01-22-200.00-0.tif_17_17_...,1,-81.850716,26.488892,"POLYGON ((-81.85357 26.48603, -81.85357 26.491...",2012
...,...,...,...,...,...,...
108,2010-2021-06-26-2021-08-25-200.00-0.tif_13_31....,0,-82.560656,28.790897,"POLYGON ((-82.56351 28.78804, -82.56351 28.793...",2010
109,2011-2020-11-23-2021-01-22-200.00-0.tif_13_31....,0,-82.673629,28.147155,"POLYGON ((-82.67649 28.14430, -82.67649 28.150...",2011
110,2012-2020-11-23-2021-01-22-200.00-0.tif_13_31....,0,-81.873589,26.408834,"POLYGON ((-81.87645 26.40598, -81.87645 26.411...",2012
111,2013-2021-07-10-2021-09-08-200.00-0.tif_13_31....,0,-82.447164,29.035679,"POLYGON ((-82.45002 29.03282, -82.45002 29.038...",2013


In [52]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import random
import shutil
import time
import warnings
import gc 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.models as models

import cv2
from matplotlib import pyplot as plt

def make_predictions(preddir, path = '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/model_best.pth.tar'):
    """
    Creates a DataFrame and Shapefile with the landuse factors extracted

    Parameters:
    preddir(str): path to the directory of images to make land use predictions on
    path(str): path to torch model weights artifacts (model_best.pth.tar file)
    """
    SEED=1
    random.seed(SEED)
    torch.manual_seed(SEED)
    cudnn.deterministic = True
    #########################
    # Define epochs and printing frequency
    START_EPOCH = 0
    #EPOCHS = 10
    PRINT_FREQ = 2500

    #Dynamically set batch size and workers
    avail_gpus = min(1, torch.cuda.device_count())
    batch_size = 100 if avail_gpus else 64
    #batch_size = 50
    TRAIN_BATCH= batch_size
    VAL_BATCH = batch_size
    WORKERS= int(os.cpu_count()-2/2)

    # Files locations of training and validation data
    # root_dir = "data"
    # TRAINDIR= os.path.join(root_dir,'train')
    # VALDIR= os.path.join(root_dir,'val')


    # check if cuda is available in this cell
    # if it is not available, you should not go forward!
    device = "cuda:0" if torch.cuda.is_available() else 'cpu'
    device = 'cpu'

    # enable algorithm optimization
    cudnn.benchmark = True

    # This cell is tagged `parameters` for papermill
    LR = 0.1
    MOMENTUM = 0.9
    WEIGHT_DECAY = 5e-4
    EPOCHS = 120

    IMG_SIZE = 224
    IMG_SIZE2 = int(round(1.15 * IMG_SIZE,0))
    imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
    imagenet_std_RGB = [0.229, 0.224, 0.225]

    IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
    }
#### Load in RESNET50 model architecture and weights from best performing model
    model = torchvision.models.resnet50()

    # Freeze all layers except for the final layer
    for param in model.parameters():
        param.requires_grad = False
        
        classes = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']

        PATH = path

        # Getting the final layer to match the number of classes
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, len(classes))


        optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM)

        checkpoint = torch.load(PATH,map_location='cpu')
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epoch = checkpoint['epoch']

        model.cpu()

        model.eval()

    # Create a folder called `predictions` in the sinkhole folder, 
    # in the `predictions` folder create a `unlabeled` subfolder, 
    # paste all images that need predictions in this folder.
    PREDDIR = preddir

    transform_val = transforms.Compose([
        transforms.Resize(IMG_SIZE2),
        transforms.CenterCrop(IMG_SIZE),
    #     transforms.Grayscale(3),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB),
    ])

    pred_dataset = ImageFolder(root=PREDDIR,transform=transform_val)

    pred_loader = torch.utils.data.DataLoader(pred_dataset, batch_size=batch_size,
                                            shuffle=False, num_workers=WORKERS) 
    with torch.no_grad():
        end = time.time()
        outputs = pd.DataFrame()
        predictions = pd.DataFrame()
        for i, (images, target) in enumerate(pred_loader):
            #print(images.shape)
            #print(i)
            ### send the images and target to cuda
            images = images.to(device)
    
            # compute output
            # output = model ??? images?
            output = model(images)
            _ , prediction = torch.max(output.cpu().detach(), dim=1)
            prob = F.softmax(output, dim=1)
            predictions = predictions.append(pd.DataFrame(prediction.cpu().numpy()))
            outputs = outputs.append(pd.DataFrame(prob.cpu().numpy()))
    # Create DataFrame & GeoDataframe
    outputs.columns = classes
    outputs.reset_index(drop=True, inplace=True)
    names = pd.DataFrame({'name':os.listdir(PREDDIR+'/unlabeled')})
    final_outputs = pd.merge(names,outputs, left_index=True, right_index=True, how='inner')
    predictions.columns = ['prediction']
    predictions.reset_index(drop=True, inplace=True)
    predictions['prediction_name'] = predictions.apply(lambda row: IDX_CLASS_LABELS[row['prediction']], axis=1)
    return  pd.merge(final_outputs,predictions, left_index=True, right_index=True, how='inner')

In [53]:
predictions = make_predictions('predictions')

In [54]:
predictions

,name,AnnualCrop,Forest,HerbaceousVegetation,Highway,Industrial,Pasture,PermanentCrop,Residential,River,SeaLake,prediction,prediction_name
0,1088-2021-11-07-2022-01-06-200.00-0.jpeg_17_17...,5.980202e-14,3.378588e-05,3.368031e-08,1.177596e-03,1.147330e-02,2.706910e-07,8.563887e-11,3.941110e-06,9.873112e-01,4.448564e-09,8,River
1,1083-2021-07-26-2021-09-24-200.00-0.jpeg_17_17...,6.552568e-10,4.421127e-04,9.890496e-01,8.411142e-07,6.272236e-10,4.337526e-12,1.166610e-07,8.660141e-09,1.247233e-03,9.260166e-03,2,HerbaceousVegetation
2,3272-2019-12-28-2020-02-26-200.00-0.jpeg_13_31...,2.209712e-16,1.154709e-06,3.918450e-07,3.311947e-04,9.985066e-01,1.143322e-12,1.241940e-13,4.574522e-09,1.160001e-03,5.807059e-07,4,Industrial
3,2343-2021-03-04-2021-05-03-200.00-0.jpeg_13_31...,7.636557e-16,4.195489e-10,1.166656e-08,8.741341e-08,9.999998e-01,2.444843e-13,3.959779e-11,2.739332e-09,4.128897e-08,6.710341e-08,4,Industrial
4,2066-2021-04-14-2021-06-13-200.00-0.jpeg_13_31...,4.011488e-12,2.988802e-01,6.606928e-01,9.158178e-05,5.386844e-07,3.893170e-21,5.975983e-12,3.071311e-09,1.059439e-05,4.032420e-02,2,HerbaceousVegetation
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,1073-2021-06-29-2021-08-28-200.00-0.jpeg_17_17...,3.157775e-18,4.418051e-10,3.672392e-08,1.938058e-05,9.998906e-01,1.249740e-16,5.502405e-13,2.734385e-07,8.968732e-05,9.788992e-08,4,Industrial
222,428-2019-12-10-2020-02-08-200.00-0.jpeg_13_31....,1.821344e-13,1.722541e-01,8.277166e-01,3.004096e-10,2.176857e-12,4.570933e-14,1.730879e-09,3.968952e-12,6.350948e-09,2.924069e-05,2,HerbaceousVegetation
223,2349-2021-09-26-2021-11-25-200.00-0.jpeg_17_17...,1.345091e-13,8.257397e-09,3.360707e-09,4.339021e-08,1.176165e-12,4.339884e-22,2.710625e-14,4.138257e-14,4.437136e-05,9.999557e-01,9,SeaLake
224,2735-2022-03-15-2022-05-14-200.00-0.jpeg_13_31...,7.368301e-08,6.956045e-02,8.302165e-01,1.160196e-05,5.902016e-06,9.472726e-13,3.777157e-06,1.291497e-09,4.519930e-02,5.500240e-02,2,HerbaceousVegetation


In [55]:
final['name'] = final.apply(lambda row: row['name'].replace('.tif','.jpeg'),axis=1)

combined = pd.merge(final, predictions, on ='name', how='left')

In [56]:
combined['imgnum'] = 0

In [57]:
combined['start_date'] = combined.apply(lambda row: row['name'].split('-')[2]+"/"+row['name'].split('-')[3]+"/"+row['name'].split('-')[1],axis=1)

In [58]:
combined.columns

Index(['name', 'label', 'centroid_x', 'centroid_y', 'geometry', 'id',
       'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
       'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake',
       'prediction', 'prediction_name', 'imgnum', 'start_date'],
      dtype='object')

In [59]:
combined

,name,label,centroid_x,centroid_y,geometry,id,AnnualCrop,Forest,HerbaceousVegetation,Highway,Industrial,Pasture,PermanentCrop,Residential,River,SeaLake,prediction,prediction_name,imgnum,start_date
0,1078-2021-02-25-2021-04-26-200.00-0.jpeg_17_17...,1,-80.084628,26.260585,"POLYGON ((-80.08749 26.25773, -80.08749 26.263...",1078,2.522005e-12,9.999354e-01,1.060090e-07,4.994671e-12,6.017314e-20,1.503631e-22,1.772614e-14,6.588980e-16,1.074310e-08,6.452300e-05,1,Forest,0,02/25/2021
1,1088-2021-11-07-2022-01-06-200.00-0.jpeg_17_17...,1,-85.594624,30.565627,"POLYGON ((-85.59748 30.56277, -85.59748 30.568...",1088,5.980202e-14,3.378588e-05,3.368031e-08,1.177596e-03,1.147330e-02,2.706910e-07,8.563887e-11,3.941110e-06,9.873112e-01,4.448564e-09,8,River,0,11/07/2021
2,429-2020-01-04-2020-03-04-200.00-0.jpeg_17_17_...,1,-82.161308,29.208182,"POLYGON ((-82.16417 29.20532, -82.16417 29.211...",429,2.864375e-05,1.364544e-10,3.386044e-25,3.439700e-14,1.341056e-18,2.133550e-30,1.461636e-22,1.407297e-11,4.115282e-03,9.958561e-01,9,SeaLake,0,01/04/2020
3,2405-2022-03-15-2022-05-14-200.00-0.jpeg_17_17...,1,-82.166159,29.174361,"POLYGON ((-82.16902 29.17150, -82.16902 29.177...",2405,6.015187e-16,9.999421e-01,2.586734e-05,5.947397e-09,4.157739e-12,8.088269e-16,8.257693e-13,8.710778e-12,1.775418e-06,3.027139e-05,1,Forest,0,03/15/2022
4,2012-2020-11-23-2021-01-22-200.00-0.jpeg_17_17...,1,-81.850716,26.488892,"POLYGON ((-81.85357 26.48603, -81.85357 26.491...",2012,1.616767e-06,2.974411e-02,6.227537e-05,3.039309e-08,2.062981e-16,1.722561e-16,4.056158e-13,2.567345e-11,1.098318e-03,9.690937e-01,9,SeaLake,0,11/23/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2010-2021-06-26-2021-08-25-200.00-0.jpeg_13_31...,0,-82.560656,28.790897,"POLYGON ((-82.56351 28.78804, -82.56351 28.793...",2010,2.093783e-18,1.073604e-13,9.358565e-15,9.408651e-06,9.999906e-01,3.262419e-18,1.007250e-13,3.616566e-10,6.837057e-08,3.035234e-12,4,Industrial,0,06/26/2021
222,2011-2020-11-23-2021-01-22-200.00-0.jpeg_13_31...,0,-82.673629,28.147155,"POLYGON ((-82.67649 28.14430, -82.67649 28.150...",2011,3.024415e-10,1.920409e-09,1.089945e-05,9.808001e-07,1.738487e-04,2.344865e-17,2.555758e-09,1.309741e-08,6.368783e-01,3.629360e-01,8,River,0,11/23/2020
223,2012-2020-11-23-2021-01-22-200.00-0.jpeg_13_31...,0,-81.873589,26.408834,"POLYGON ((-81.87645 26.40598, -81.87645 26.411...",2012,2.864375e-05,1.364544e-10,3.386044e-25,3.439700e-14,1.341056e-18,2.133550e-30,1.461636e-22,1.407297e-11,4.115282e-03,9.958561e-01,9,SeaLake,0,11/23/2020
224,2013-2021-07-10-2021-09-08-200.00-0.jpeg_13_31...,0,-82.447164,29.035679,"POLYGON ((-82.45002 29.03282, -82.45002 29.038...",2013,2.353373e-08,6.308874e-05,8.861153e-03,5.339440e-08,3.373386e-08,4.891665e-13,6.528018e-09,1.359047e-10,4.597301e-02,9.451028e-01,9,SeaLake,0,07/10/2021


In [60]:
final_final = combined[['name','imgnum', 'label', 'id','centroid_x', 'centroid_y','start_date','geometry','AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
       'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake',
       'prediction', 'prediction_name']]

In [61]:
final_final.columns = ['name','imgnum', 'label', 'ID','lon', 'lat','start_date','geometry','AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
       'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake',
       'prediction', 'prediction_name']

In [62]:
final_final.to_csv('model_sh_attr_1_group_2_group_0.csv',index=None)

In [63]:
!zip -r /content/jpegs200at64_10yr_1.zip /content/jpegs200at64_10yr

  adding: content/jpegs200at64_10yr/ (stored 0%)
  adding: content/jpegs200at64_10yr/1088-2021-11-07-2022-01-06-200.00-0.jpeg_17_17_sinkhole.jpeg (deflated 43%)
  adding: content/jpegs200at64_10yr/1083-2021-07-26-2021-09-24-200.00-0.jpeg_17_17_sinkhole.jpeg (deflated 43%)
  adding: content/jpegs200at64_10yr/3272-2019-12-28-2020-02-26-200.00-0.jpeg_13_31.jpeg (deflated 3%)
  adding: content/jpegs200at64_10yr/2343-2021-03-04-2021-05-03-200.00-0.jpeg_13_31.jpeg (deflated 2%)
  adding: content/jpegs200at64_10yr/2066-2021-04-14-2021-06-13-200.00-0.jpeg_13_31.jpeg (deflated 3%)
  adding: content/jpegs200at64_10yr/3934-2021-10-13-2021-12-12-200.00-0.jpeg_17_17_sinkhole.jpeg (deflated 2%)
  adding: content/jpegs200at64_10yr/2057-2020-05-31-2020-07-30-200.00-0.jpeg_17_17_sinkhole.jpeg (deflated 2%)
  adding: content/jpegs200at64_10yr/3001-2021-02-17-2021-04-18-200.00-0.jpeg_17_17_sinkhole.jpeg (deflated 2%)
  adding: content/jpegs200at64_10yr/1080-2020-12-08-2021-02-06-200.00-0.jpeg_13_31.jpeg 